# Week 1: CUDA operations in PyTorch. Introduction to benchmarking.
In this seminar, we'll learn a bit more about the things one needs to keep in mind when using GPU computations, both in general and in PyTorch. We'll also see a couple of examples on benchmarking code in Python; again, there are some caveats with CUDA.

First, let's import PyTorch and get some information about the currently used device:

In [1]:
import torch

torch.cuda.is_available()

False

In [2]:
torch.cuda.get_device_properties(0)

AssertionError: Torch not compiled with CUDA enabled

## Memory allocation
As discussed in the lecture, GPU memory is separate from the CPU memory and needs to be explicitly allocated, triggering a host-device synchronization. PyTorch uses a caching memory allocator to repurpose already available but unused memory fragments.

See this example: we allocate the memory inside the function scope, so the tensor is deleted as soon as the scope is left:

In [3]:
def allocate_empty_tensor(dim_size):
    a = torch.empty(4096, dim_size, dtype=torch.float32, device="cuda")

In [4]:
allocate_empty_tensor(2048)

Printing the allocated memory size gives us an expected number of zero bytes:

In [5]:
torch.cuda.memory_allocated(0)

0

However, the GPU memory is still in use by the process; this is shown by `torch.cuda.memory_reserved` or `nvidia-smi` from the terminal. As a result, working in a shared GPU environment can leave a lot of unused yet allocated memory.

In [6]:
torch.cuda.memory_reserved()

33554432

In [7]:
!nvidia-smi

Tue Jan 17 22:16:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W /  70W |    134MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Let's clear the cache now and see what happens:

In [8]:
torch.cuda.empty_cache()
torch.cuda.memory_reserved()

0

Note that this operation triggers a CPU-GPU synchronization, and thus using it in your code can significantly hurt the performance. It's almost always better to carefully manage the lifetime of your GPU tensors and avoid excessive allocations than to empty the cache.

Now, let's see how this cache is reused by allocating two tensors in a row: first a larger one, then a smaller one.

In [9]:
allocate_empty_tensor(2048)
torch.cuda.memory_reserved()

33554432

In [10]:
allocate_empty_tensor(1024)
torch.cuda.memory_reserved()

33554432

As expected, we reuse the cache, since the array fits into the allocated chunk.

However, if we attempt to do this with a larger tensor (3072 elements in the second dimension instead of 2048), we observe something different:

In [11]:
allocate_empty_tensor(3072)
torch.cuda.memory_reserved()

83886080

What happened? The chunk of memory that was allocated for a 4096x2048 array did not fit a tensor of size 4096x3072: thus, PyTorch needed to allocate an additional segment of a sufficient size while keeping the previous one allocated (in case the user creates a smaller tensor later at some point).

In practice, this means that if your code is dealing with tensors of dynamic size (changing batch sizes, sequence lengths or image resolutions), it is recommended to first warm up the cache by allocating tensors for the largest expected input. Otherwise, in the worst case you allocate a quadratic amount of memory with respect to the largest input size instead of a linear one.

You can also view more detailed allocation statistics by running `torch.cuda.memory_stats()`

In [12]:
memory_stats = torch.cuda.memory_stats()
print(memory_stats["active.all.allocated"])
print(memory_stats["active.all.current"])
print(memory_stats["active.all.peak"])
print(memory_stats["reserved_bytes.all.current"])

4
0
1
83886080


In [13]:
torch.cuda.empty_cache()
print(torch.cuda.memory_stats()["reserved_bytes.all.current"])

0


## Benchmarking intro
The simplest way to check the performance impact of any change is to compare the runtime of code with and without it. Here, we will consider a simple way of doing this in Python.

First, let's define two functions that compute a batched version of a dot product for two matrices:

In [14]:
def batched_dot_mul_sum(a, b):
    """Computes batched dot by multiplying and summing"""
    return a.mul(b).sum(-1)


def batched_dot_bmm(a, b):
    """Computes batched dot by reducing to bmm"""
    a = a.reshape(-1, 1, a.shape[-1])
    b = b.reshape(-1, b.shape[-1], 1)
    return torch.bmm(a, b).flatten(-3)


# Input for benchmarking
x = torch.randn(10000, 64)

# Ensure that both functions compute the same output
assert batched_dot_mul_sum(x, x).allclose(batched_dot_bmm(x, x))

To conduct microbenchmarks by running the code hundreds of times and measuring the average execution time, you can use the built-in [timeit](https://docs.python.org/3/library/timeit.html) module. Simply create a Timer object with relevant arguments and call `.timeit()`:

In [15]:
import timeit

t0 = timeit.Timer(
    stmt="batched_dot_mul_sum(x, x)",
    setup="from __main__ import batched_dot_mul_sum",
    globals={"x": x},
)

t1 = timeit.Timer(
    stmt="batched_dot_bmm(x, x)",
    setup="from __main__ import batched_dot_bmm",
    globals={"x": x},
)

print(f"mul_sum(x, x):  {t0.timeit(100) / 100 * 1e6:>5.1f} us")
print(f"bmm(x, x):      {t1.timeit(100) / 100 * 1e6:>5.1f} us")

mul_sum(x, x):  605.6 us
bmm(x, x):      888.7 us


In IPython, there exist line and cell timeit [magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cell-magics). They can be more convenient for smaller cases but allow you a bit less control over the setup.

In [16]:
%timeit batched_dot_mul_sum(x, x)

526 µs ± 24.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
%timeit batched_dot_bmm(x, x)

892 µs ± 37.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


[torch.utils.benchmark](https://pytorch.org/docs/stable/benchmark_utils.html) copies the API of timeit while helping the user avoid common mistakes. 

First, let's run it on the same code and data:

In [18]:
import torch.utils.benchmark as benchmark

t0 = benchmark.Timer(
    stmt="batched_dot_mul_sum(x, x)",
    setup="from __main__ import batched_dot_mul_sum",
    globals={"x": x},
)

t1 = benchmark.Timer(
    stmt="batched_dot_bmm(x, x)",
    setup="from __main__ import batched_dot_bmm",
    globals={"x": x},
)

print(t0.timeit(100))
print(t1.timeit(100))

batched_dot_mul_sum(x, x)
setup: from __main__ import batched_dot_mul_sum
  554.54 us
  1 measurement, 100 runs , 1 thread
batched_dot_bmm(x, x)
setup: from __main__ import batched_dot_bmm
  843.06 us
  1 measurement, 100 runs , 1 thread


In [19]:
# in addition, we can set the number of threads for CPU computations
num_threads = torch.get_num_threads()
print(f"Benchmarking on {num_threads} threads")

t0 = benchmark.Timer(
    stmt="batched_dot_mul_sum(x, x)",
    setup="from __main__ import batched_dot_mul_sum",
    globals={"x": x},
    num_threads=num_threads,
    label="Multithreaded batch dot",
    sub_label="Implemented using mul and sum",
)

t1 = benchmark.Timer(
    stmt="batched_dot_bmm(x, x)",
    setup="from __main__ import batched_dot_bmm",
    globals={"x": x},
    num_threads=num_threads,
    label="Multithreaded batch dot",
    sub_label="Implemented using bmm",
)

print(t0.timeit(100))
print(t1.timeit(100))

Benchmarking on 1 threads
Multithreaded batch dot: Implemented using mul and sum
setup: from __main__ import batched_dot_mul_sum
  543.99 us
  1 measurement, 100 runs , 1 thread
Multithreaded batch dot: Implemented using bmm
setup: from __main__ import batched_dot_bmm
  859.42 us
  1 measurement, 100 runs , 1 thread


In [20]:
# we can change it globally for PyTorch and measure the impact
prev_num_threads = num_threads
torch.set_num_threads(2)

num_threads = torch.get_num_threads()
print(f"Benchmarking on {num_threads} threads")

t0 = benchmark.Timer(
    stmt="batched_dot_mul_sum(x, x)",
    setup="from __main__ import batched_dot_mul_sum",
    globals={"x": x},
    num_threads=num_threads,
    label="Multithreaded batch dot",
    sub_label="Implemented using mul and sum",
)

t1 = benchmark.Timer(
    stmt="batched_dot_bmm(x, x)",
    setup="from __main__ import batched_dot_bmm",
    globals={"x": x},
    num_threads=num_threads,
    label="Multithreaded batch dot",
    sub_label="Implemented using bmm",
)

print(t0.timeit(100))
print(t1.timeit(100))
# in this case, we don't get any speedup, likely due to the overhead

torch.set_num_threads(prev_num_threads)

Benchmarking on 2 threads
Multithreaded batch dot: Implemented using mul and sum
setup: from __main__ import batched_dot_mul_sum
  898.22 us
  1 measurement, 100 runs , 2 threads
Multithreaded batch dot: Implemented using bmm
setup: from __main__ import batched_dot_bmm
  1.36 ms
  1 measurement, 100 runs , 2 threads


In [21]:
# by the way, what CPU do we have?
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

## Benchmarking GPU code
As we discussed in the lecture, CUDA kernel execution and PyTorch GPU operations are asynchronous. 
This means that it is possible to launch several kernels in a row before receiving results from the first one. As a consequence, naive benchmarking without synchronization is likely to give you unrealistic results.

Let's the same example, but with slightly larger matrices on the GPU:

In [22]:
import timeit

x = torch.randn(10000, 1024, device="cuda")

t0 = timeit.Timer(
    stmt="batched_dot_mul_sum(x, x)",
    setup="from __main__ import batched_dot_mul_sum",
    globals={"x": x},
)

t1 = timeit.Timer(
    stmt="batched_dot_bmm(x, x)",
    setup="from __main__ import batched_dot_bmm",
    globals={"x": x},
)

# Ran each twice to show difference before/after warmup
print(f"mul_sum(x, x):  {t0.timeit(100) / 100 * 1e6:>5.1f} us")
print(f"mul_sum(x, x):  {t0.timeit(100) / 100 * 1e6:>5.1f} us")
print(f"bmm(x, x):      {t1.timeit(100) / 100 * 1e6:>5.1f} us")
print(f"bmm(x, x):      {t1.timeit(100) / 100 * 1e6:>5.1f} us")

mul_sum(x, x):   91.7 us
mul_sum(x, x):   23.3 us
bmm(x, x):      32066.8 us
bmm(x, x):       29.2 us


First, we see that the difference between the first and the second runs of timeit is quite noticeable, which happens because of initializing the CUDA context and loading [cuBLAS](https://docs.nvidia.com/cuda/cublas/index.html) — a CUDA library for accelerated linear algebra.

Second, the runtimes of two methods seem too small. 

Let's run the same test with `torch.utils.benchmark` to see a different set of results.

In [23]:
t0 = benchmark.Timer(
    stmt="batched_dot_mul_sum(x, x)",
    setup="from __main__ import batched_dot_mul_sum",
    globals={"x": x},
)

t1 = benchmark.Timer(
    stmt="batched_dot_bmm(x, x)",
    setup="from __main__ import batched_dot_bmm",
    globals={"x": x},
)

# Run only once since benchmark module does warmup for us
print(t0.timeit(100))
print(t1.timeit(100))

batched_dot_mul_sum(x, x)
setup: from __main__ import batched_dot_mul_sum
  501.08 us
  1 measurement, 100 runs , 1 thread
batched_dot_bmm(x, x)
setup: from __main__ import batched_dot_bmm
  225.33 us
  1 measurement, 100 runs , 1 thread


Now we have a more realistic set of measurements, which is caused by explicitly triggering the CPU-GPU synchronization and awaiting the results after launching the multiplication. The runtime is comparable to what we had in a previous set of benchmarks, but pay attention that now the matrices being multiplied are 16 times bigger.

Let's implement microbenchmarking by ourselves using nothing but [`time.perf_counter()`](https://docs.python.org/3/library/time.html#time.perf_counter) and CUDA methods given by PyTorch. First, we'll run `batched_dot_mul_sum` on the GPU with and without synchronization and see how this affects the results:

In [24]:
from time import perf_counter

import numpy as np

execution_times = []

for _ in range(100):
    start_time = perf_counter()
    batched_dot_mul_sum(x, x)
    execution_times.append(perf_counter() - start_time)

np.mean(execution_times)

5.138728001838899e-05

Compare this with the result that does not compute anything:

In [25]:
execution_times = []

for _ in range(100):
    start = perf_counter()
    execution_times.append(perf_counter() - start)

np.mean(execution_times)

1.1086999620602001e-07

Finally, let's explicitly call `torch.cuda.synchronize` at the end of each iteration to see the difference.

In [26]:
execution_times = []

for _ in range(100):
    start_time = perf_counter()
    batched_dot_mul_sum(x, x)
    torch.cuda.synchronize()
    execution_times.append(perf_counter() - start_time)

np.mean(execution_times)

0.0007799099600060799

The same thing applies to the actual models. Let's take `Linear` as the simplest example:

In [27]:
x = torch.randn(10000, 512, device="cuda")
linear = torch.nn.Linear(512, 1024, bias=False, device="cuda")
N_ITERS = 200

execution_times = []

for _ in range(N_ITERS):
    start = perf_counter()
    result = linear(x)
    execution_times.append(perf_counter() - start)

np.mean(execution_times)

8.6860699996123e-05

In [28]:
execution_times = []

for _ in range(N_ITERS):
    start = perf_counter()
    result = linear(x)
    torch.cuda.synchronize()
    execution_times.append(perf_counter() - start)

np.mean(execution_times)

0.004536991950005813

In [29]:
execution_times = []

start = perf_counter()
for _ in range(N_ITERS):
    result = linear(x)
torch.cuda.synchronize()

(perf_counter() - start) / N_ITERS

0.002294177774999753

Of course, `torch.utils.benchmark` also works:

In [30]:
t0 = benchmark.Timer(stmt="linear(x)", globals={"x": x, "linear": linear})
print(t0.timeit(100))

linear(x)
  2.24 ms
  1 measurement, 100 runs , 1 thread


## CUDA Streams
To execute several operations concurrently, you may use CUDA streams in PyTorch. Below, you can see an example of their usage.

However, as you will see in the matrix multiplication example, the usefulness of streams is quite limited: the concurrent kernels need to underutilize the GPU and yet take long enough to compute. It might be a good idea if you are trying to compute some expression and fetch data for the next computation simultaneously though.

In [31]:
cuda = torch.device("cuda")
s = torch.cuda.Stream()  # Create a new stream.
A = torch.empty((100, 100), device=cuda).normal_(0.0, 1.0)
with torch.cuda.stream(s):
    # sum() may start execution before normal_() finishes!
    B = torch.sum(A)

In [32]:
s1 = torch.cuda.Stream()
s2 = torch.cuda.Stream()
# Initialize cuda tensors here. E.g.:
A = torch.rand(1000, 1000, device="cuda")
B = torch.rand(1000, 1000, device="cuda")
# Wait for the above tensors to initialize.
torch.cuda.synchronize()

execution_times = []

for _ in range(100):
    start = perf_counter()
    with torch.cuda.stream(s1):
        C = torch.mm(A, A)
    with torch.cuda.stream(s2):
        D = torch.mm(B, B)
    # Wait for C and D to be computed.
    torch.cuda.synchronize()
    execution_times.append(perf_counter() - start)

np.mean(execution_times)

0.001015964859984706

In [33]:
# next, let's compute C and D sequentially

execution_times = []

for _ in range(100):
    start = perf_counter()
    C = torch.mm(A, A)
    D = torch.mm(B, B)
    # Wait for C and D to be computed.
    torch.cuda.synchronize()
    execution_times.append(perf_counter() - start)

np.mean(execution_times)
# the speed is even higher in this case

0.0009575643899643183

## Debugging asynchronous code
Finding sources of errors in GPU-reliant code in PyTorch can also be difficult. Let's see this on a standard example of an off-by-one error and incorrect index for the embedding layer:

In [34]:
%%writefile incorrect_index.py
import torch
import torch.nn as nn

embedding = nn.Embedding(1024,32).to('cuda')
# 1024 > 1023 (largest index in the created embedding layer)
input = torch.full((1,1),1024,dtype=torch.long, device='cuda')

# out-of-bounds access
embedding_for_index = embedding(input)

result = torch.sigmoid(embedding_for_index)
loss = result.sum()
print(loss.item())
print(loss)

Overwriting incorrect_index.py


If we attempt to run the code as is, we'll see an error after the point in which we trigger the CPU-GPU synchronization (`loss.item`). This gives us no clues about what exactly caused an error:

In [35]:
!python incorrect_index.py

../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [5,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [6,0,0

(though an experienced person might get a hint from the C++ failed assertions printed before the error)

There are two ways to find the actual line that caused an exception:

* First, you can just move everything to CPU. This is a valid approach, but it involves making changes to your code or input arguments and sometimes can be difficult (for example, if the error occurs after a long chain of operations).

* Second, you may use the `CUDA_LAUNCH_BLOCKING` environment variable when starting your code. This will force synchronization for all GPU-related operations, making your code slower but allowing to see the exact source of the error.

In [36]:
!CUDA_LAUNCH_BLOCKING=1 python incorrect_index.py

../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [5,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [0,0,0], thread: [6,0,0

## Precision of floating point operations
Let's have a look at this code snippet and its results. In essence, it computes the sixth power of a random matrix (with a fixed seed) on two different devices.

In [37]:
torch.manual_seed(1337)
x = torch.randn(5000, 5000)
torch.use_deterministic_algorithms(False)


def matrix_power(x):
    y = x @ x @ x @ x @ x @ x
    return (y).sum().item()


print(matrix_power(x))
print(matrix_power(x.cuda()))

27654770130944.0
27654807879680.0


(in case you are wondering, `torch.use_deterministic_algorithms(True)` won't help)

If we do the same using numpy (in two ways), we also get a different result:

In [38]:
print(matrix_power(x.numpy()))
np.linalg.matrix_power(x.numpy(), 6).sum()

27654791102464.0


27654774000000.0

Takeaway: numerical precision of floating point computations can vary between libraries, environments and devices, and from the user side, it is often hard to resolve this issue altogether. Usually, this happens due to a different summation order in code or due to inherent nondeterminism of hardware.

However, note that the relative error is small enough, which makes such blatant discrepancies less of a problem in regular deep learning code.

# Bonus (2/10 points for the first home assignment block)
Solve exercises in the notebook from https://github.com/srush/GPU-Puzzles. You will earn the number of points corresponding to the fraction of completed exercises.